This notebook covers:
* Regex basics
* Web scraping with BeautifulSoup

In [ ]:
import pandas as pd
import numpy as np

## Regular expressions

In [ ]:
from IPython.display import Image

I have talked about some basic regex functionality which is taken from this excellent post

https://www.machinelearningplus.com/python/python-regex-tutorial-examples/

In [ ]:
import re

In [ ]:
Image("../imgs/regex.png")

A regex pattern is a special language used to represent generic text, numbers or symbols so it can be used to extract texts that conform to that pattern.

Here the '\s' matches any whitespace character. By adding a '+' notation at the end will make the pattern match at least 1 or more spaces. So, this pattern will match even tab '\t' characters as well.

In [ ]:
regex = re.compile('\s+')

**Splitting a string using regex**

In [ ]:
text = "Hello World.   Regex is awesome"

In [ ]:
" ".join(text.split())

In [ ]:
regex.split(text)

Another way but regex is generally the better one

In [ ]:
re.split('\s', text)

**re.findall**

the findall method extracts all occurrences of the pattern

 `'\d'` is a regular expression which matches any digit

In [ ]:
text = "101 howard street, 246 mcallister street"

In [ ]:
regex_num = re.compile('\d+')  #one or more digits

In [ ]:
regex_num.findall(text)

In [ ]:
regex_num.split(text)

In [ ]:
re.findall('\d+', text)

**re.search() vs re.match()**

`regex.search()` returns a particular match object that contains the starting and ending positions of the **first occurrence of the pattern**.

Likewise, `regex.match()` also returns a match object. But the difference is, it requires the pattern to be present at the **beginning of the text itself**.

In [ ]:
text2 = "MAT 20567567576  Mathematics 189"

In [ ]:
m = regex_num.match(text2)
m

In [ ]:
m.group()

In [ ]:
m.start()  #returns the index of the starting

In [ ]:
s = regex_num.search(text2)
s

In [ ]:
s.group()

**Substituting one text by another using `regex.sub()`**

In [ ]:
text = """101   COM \t  Computers
205   MAT \t  Mathematics
189   ENG  \t  English"""

In [ ]:
regex = re.compile('\s+')

In [ ]:
regex.sub(' ', text)  #it replaces the regular expression by ' '

In [ ]:
# get rid of all extra spaces except newline
regex = re.compile('((?!\n)\s+)')
print(regex.sub(' ', text))

**combining regex pattern**

In [ ]:
# define the course text pattern groups and extract
course_pattern = '([0-9]+)\s*([A-Z]{3})\s*([A-Za-z]{4,})'
re.findall(course_pattern, text)

**greedy regex**

The default behavior of regular expressions is to be greedy. That means it tries to extract as much as possible until it conforms to a pattern even when a smaller part would have been syntactically sufficient.

In [ ]:
text = "< body>Regex Greedy Matching Example < /body>"
re.findall('<.*>', text)

it should have stopped at first > but it didn't. For extracting only the smaller portions:

Lazy matching, on the other hand, ‘takes as little as possible’. This can be effected by adding a `?` at the end of the pattern.

In [ ]:
re.findall('<.*?>', text)

In [ ]:
s = re.search('<.*?>', text)  #getting only the first one

In [ ]:
s.group()

In [ ]:
text = '01, Jan 2015'

In [ ]:
print(re.findall('\d{3}', text))

**matching word boundaries**

Word boundaries `\b` are commonly used to detect and match the beginning or end of a word. That is, one side is a word character and the other side is whitespace and vice versa.

For example, the regex \btoy will match the ‘toy’ in ‘toy cat’ and not in ‘tolstoy’. In order to match the ‘toy’ in ‘tolstoy’, you should use toy\b

Can you come up with a regex that will match only the first ‘toy’ in ‘play toy broke toys’? (hint: \b on both sides)

Likewise, `\B` will match any non-boundary.

For example, \Btoy\B will match ‘toy’ surrounded by words on both sides, as in, ‘antoynet’.

In [ ]:
re.findall(r'\btoy\b', 'play toy broke toys')

In [ ]:
re.findall(r'\btoy', 'play toy broke toys')

In [ ]:
re.findall(r'toy\b', 'play toy broke toys')

In [ ]:
re.findall(r'\Btoy\b', 'playtoy broke toys')

In [ ]:
re.findall(r'\Btoy\B', 'playtoybroke toys')

In [ ]:
re.findall(r'\btoy', 'playtoybroke toys')

**Practice regex examples**

In [ ]:
emails = """zuck26@facebook.com
page33@google.com
jeff42@amazon.com"""

desired_output = [('zuck26', 'facebook', 'com'), ('page33', 'google', 'com'),
                  ('jeff42', 'amazon', 'com')]

In [ ]:
regex = re.compile('([\w]+)@([\w]+).([\w]+)')

In [ ]:
regex.findall(emails)

2. Retrieve all the words starting with ‘b’ or ‘B’ from the following text.

In [ ]:
text = """Betty bought a bit of butter, 
But the butter was so bitter, So she bought
some better butter, To make the bitter butter better."""

In [ ]:
regex = re.compile('([$bB]\w+)')

In [ ]:
regex.findall(text)

In [ ]:
sentence = """A, very   very; irregular_sentence"""
desired_output = "A very very irregular sentence"

In [ ]:
regex = re.compile('[,\s;_]+')

In [ ]:
' '.join(regex.split(sentence))

In [ ]:
tweet = '''Good advice! RT @TheNextWeb: What I would do differently if I was learning to code today http://t.co/lbwej0pxOd cc: @garybernhardt #rstats'''

In [ ]:
desired_output = 'Good advice What I would do differently if I was learning to code today'

In [ ]:
def clean_tweet(tweet):
    tweet = re.sub('http\S+\s*', '', tweet)  # remove URLs
    tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
    tweet = re.sub('#\S+', '', tweet)  # remove hashtags
    tweet = re.sub('@\S+', '', tweet)  # remove mentions
    tweet = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""),
                   '', tweet)  # remove punctuations
    tweet = re.sub('\s+', ' ', tweet)  # remove extra whitespace
    return tweet


print(clean_tweet(tweet))

# Web Scraping with BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import requests

In [ ]:
def download_html(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    return html

In [ ]:
url = 'https://www.imdb.com/search/title?release_date=2018&sort=boxoffice_gross_us,desc&start=1'
html = download_html(url)

soup = BeautifulSoup(html, 'lxml') #html.parser

Since above code extracts all data on the first page, below code is run only to extract movie information on it.

In [ ]:
movie_blocks = soup.findAll('div',  {'class':'lister-item-content'})

In [ ]:
movie_blocks[0]

BeautifulSoup.find_all(arguments) returns a list of BeautifulSoup objects. These are all occurrences matching the arguments. If there are no matches, method returns empty list. This is obviously used, when you cannot identify it right away and have to do some more digging before you get to the data you want.

> Let's examine one of the extracted block to identify the elements that we need to scrape.

In [ ]:
movie_blocks[0].find('span', {'class': 'lister-item-year'}).contents

In [ ]:
import re
year = re.compile("\d+")
year.search(movie_blocks[0].find('span',{'class': 'lister-item-year'}).contents[0]).group()

In [ ]:
mname = movie_blocks[0].find('a').get_text() # Name of the movie

m_reyear = int(movie_blocks[0].find('span',{'class': 'lister-item-year'}).contents[0][1:-1]) # Release year

m_rating = float(movie_blocks[0].find('div',{'class':'inline-block ratings-imdb-rating'}).get('data-value')) #rating

m_mscore = float(movie_blocks[0].find('span',{'class':'metascore favorable'}).contents[0].strip()) #meta score

m_votes = int(movie_blocks[0].find('span',{'name':'nv'}).get('data-value')) # votes

print("Movie Name: " + mname,
      "\nRelease Year: " + str(m_reyear),
      "\nIMDb Rating: " + str(m_rating),
      "\nMeta score: " + str(m_mscore),
      "\nVotes: " + '{:,}'.format(m_votes)

)

In [ ]:
def scrape_mblock(movie_block):
    
    movieb_data ={}
  
    try:
        movieb_data['name'] = movie_block.find('a').get_text() # Name of the movie
    except:
        movieb_data['name'] = None

    try:    
        movieb_data['year'] = str(movie_block.find('span',{'class': 'lister-item-year'}).contents[0][1:-1]) # Release year
    except:
        movieb_data['year'] = None

    try:
        movieb_data['rating'] = float(movie_block.find('div',{'class':'inline-block ratings-imdb-rating'}).get('data-value')) #rating
    except:
        movieb_data['rating'] = None
    
    try:
        movieb_data['m_score'] = float(movie_block.find('span',{'class':'metascore favorable'}).contents[0].strip()) #meta score
    except:
        movieb_data['m_score'] = None

    try:
        movieb_data['votes'] = int(movie_block.find('span',{'name':'nv'}).get('data-value')) # votes
    except:
        movieb_data['votes'] = None

    return movieb_data

> Then I create the below function to scrape all movie blocks within a single search result page

In [ ]:
def scrape_m_page(movie_blocks):
    
    page_movie_data = []
    num_blocks = len(movie_blocks)
    
    for block in range(num_blocks):
        page_movie_data.append(scrape_mblock(movie_blocks[block]))
    
    return page_movie_data

> Now we built functions to extract all movie data from a single page.

Next function will be created to iterate the above made function through all pages of the search result untill we scrape data for the targeted number of movies

In [ ]:
import time     
import random as ran
def scrape_this(link,t_count):
    
    #from IPython.core.debugger import set_trace

    base_url = link
    target = t_count
    
    current_mcount_start = 0
    current_mcount_end = 0
    remaining_mcount = target - current_mcount_end 
    
    new_page_number = 1
    
    movie_data = []
    
    
    while remaining_mcount > 0:

        url = base_url + str(new_page_number)
        
        #set_trace()
        
        source = download_html(url)
        soup = BeautifulSoup(source, 'html.parser') # lxml
        
        movie_blocks = soup.findAll('div',{'class':'lister-item-content'})
        
        movie_data.extend(scrape_m_page(movie_blocks))   
        
        current_mcount_start = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[0])

        current_mcount_end = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[1].split(" ")[0])

        remaining_mcount = target - current_mcount_end
        
        print('\r' + "currently scraping movies from: " + str(current_mcount_start) + " - "+str(current_mcount_end), "| remaining count: " + str(remaining_mcount), flush=True, end ="")
        
        new_page_number = current_mcount_end + 1
        
        time.sleep(ran.randint(0, 10))
    
    return movie_data

> Finally, we have put together all functions created above to scrape the top 150 movies on the list

In [ ]:
import pandas as pd 
base_scraping_link = "https://www.imdb.com/search/title?release_date=2018-01-01,2018-12-31&sort=boxoffice_gross_us,desc&start="

top_movies = 150 #input("How many movies do you want to scrape?")
films = []

movies = scrape_this(base_scraping_link,int(top_movies))

print('\r'+"List of top " + str(top_movies) +" movies:" + "\n", end="\n")
movies=pd.DataFrame(movies)
movies

In [ ]:
movies.to_csv('../datasets/movies.csv', index=False)